In [1]:
import os
import time
import numpy as np
import cv2
from glob import glob
from matplotlib import pyplot
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
img_h = 64
img_w = 64
img_c = 3

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
def load_image(image_Path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_png(img)
    img = tf.cast(img,tf.float32)
    img = (img-127.5)/127.5
    return img

In [5]:
# for top,dirs, files in os.walk("/kaggle/input/animefacedataset/images"):
#     for file in files:
#         load_image(os.path.join(top,file))

In [6]:
def tf_dataset(images_path, batch_size):
    ds = tf.data.Dataset.from_tensor_silces(images_path)
    ds = ds.shuffle(buffer_size = 1000).map(load_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return ds

In [7]:
def build_generator(latent_dim):
    noise = L.Input((latent_dim),name = "noise_input")
    
    x = L.Dense(256)(noise)
    x = L.LeakyReLU(0.2)(x)
    
    x = L.Dense(1024)(noise)
    x = L.LeakyReLU(0.2)(x)
    
    x = L.Dense(4096)(noise)
    x = L.LeakyReLU(0.2)(x)
    
    x = L.Dense(img_h*img_w*img_c)(x)
    x = L.LeakyReLU(0.2)(x)
    
    x = L.Reshape((img_h,img_w,img_c))(x)
    
    fake_output = L.Activation("tanh")(x)
    
    return Model(noise, fake_output, name = "generator")

In [8]:
def build_discriminator():
    inputs = L.Input((img_h,img_w,img_c), name = "disc_input")
    
    x = L.Flatten()(inputs)
    
    x = L.Dense(4096)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)
    
    x = L.Dense(1024)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)
    
    x = L.Dense(256)(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Dropout(0.3)(x)
    
    x = L.Dense(1)(x)
    
    return Model(inputs, x, name = "discriminator")

In [9]:
@tf.function
def train_step(real_images, latent_dim, generator, discriminator, g_opt,d_opt):
    batch_size= tf.shape(real_images)[0]
    bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits = True, label_smoothing = 0.1)
    
    noise = tf.random.normal([batch_size, latent_dim])
    for _ in range(2):
        with tf.GradientTape() as dtape:
            generated_images = generator(noise, training = True)
            
            real_output = discriminator(real_images, training = True)
            fake_output = discriminator(generated_images, training = True)
            
            d_real_loss = bce_loss(tf.ones_like(real_output), real_output)
            d_fake_loss = bce_Loss(tf.zeros_like(fake_output), fake_output)
            d_loss = d_real_loss + d_fake_loss
            
            d_grad = dtape.gradient(d_loss, discriminator.trainable_variables)
            d_opt.apply_gradients(zip(d_grad,discriminator.trainable_variables))
            
        with tf.GradientType() as gtape:
            generated_images = generator(noise, training = True)
            fake_output = discriminator(generated_images, training = True)
            
            g_loss = bce_loss(tf.ones_like(fake_output), fake_output)
            
            g_grad = gtape.gradient(g_loss,generator.trainable_variables)
            g_opt.apply_gradients(zip(g_grad, generator.trainable_variables))
            
        return d_loss, g_loss

In [10]:
def save_plot(examples, epoch, n):
    n = int(n)
    examples = (examples+1)/2.0
    examples = examples*255
    file_name = f"samples/generated_plot_apoch-{epoch+1}.png"
    
    cat_image = None
    for i in range(n):
        start_idx = i*n
        end_idx = (i+1)*n
        
        image_List = examples[start_idx:end_idx]
        if i == 0:
            cat_image = np.concatenate(image_list, axis = 1)
        else:
            tmp = np.concatenate(image_list, axis = 1)
            cat_image = np.concatenate([cat_image, tmp], axis = 0)
        
        cv2.imwrite(file_name,cat_image)
    

In [11]:
batch_size = 128
latent_dim = 64
num_epocs = 1000
n_samples = 100

images_path = glob("/kaggle/input/animefacedataset/images/*.png")
print(f"images :{len(images_path)}")

create_dir("samples")
create_dir("saved_model")

g_model = build_generator(latent_dim)
d_model = build_discriminator()

g_model.summary()
d_model.summary()

images :0
Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 noise_input (InputLayer)    [(None, 64)]              0         
                                                                 
 dense_2 (Dense)             (None, 4096)              266240    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 4096)              0         
                                                                 
 dense_3 (Dense)             (None, 12288)             50343936  
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 12288)             0         
                                                                 
 reshape (Reshape)           (None, 64, 64, 3)         0         
                                                                 
 activation (Activation)     (None, 64, 64, 3) 

In [12]:
# d_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_l = 0.5)
# g_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_l = 0.5)

# images_dataset = tf.dataset(images_path, batch_size)
# seed = np.random.normal(size = (n_samples, latent_dim))

# for epoch in range(num_epochs):
#     start = time.time()
    
#     d_loss = 0.0
#     g_loss = 0.0
    
#     for image_batch in images_dataset:
#         d_batch_loss, g_batch_loss = train_step(image_batch, latent_dim, g_model, d_model, g_optimizer, d_optimizer)
#         d_loss +=  d_batch_loss
#         g_loss += g_batch_loss
    
#     d_loss = d_loss/len(images_dataset)
#     g_loss = g_loss/len(images_dataset)
    
#     g_model.save("saved_model/g_model.h5")
#     d_model.save("saved_model/d_model.h5")
    
#     examples = g_model.predict(seed, verbose = 0)
#     save_plot(examples, epoch, np.sqrt(n_samples))
    
#     time_taken  = time.time()-start
#     print(f"[{epoch+1:1.0f}/{num_epochs}] {time_taken:2.2f}s - d_loss: {d_loss:1.4f}- g_loss: {g_loss:1.4f}")